# IMDB 영화 리뷰 감정 분석 (사전 훈련된 GloVe 임베딩 + 양방향 LSTM)

이 노트북은 사전 훈련된 GloVe 임베딩을 사용하여 IMDB 영화 리뷰 감정 분석을 수행합니다.

## 주요 특징
- **데이터**: IMDB 영화 리뷰 (긍정/부정 이진 분류)
- **임베딩**: 사전 훈련된 GloVe 6B.100d 임베딩 사용
- **모델**: 양방향 LSTM (Bidirectional LSTM)
- **가중치 고정**: trainable=False로 사전 훈련된 가중치 보존
- **평가**: 훈련/검증/테스트 데이터셋으로 성능 평가

## 워크플로우
1. 데이터 다운로드 및 전처리
2. 데이터셋 로드 및 텍스트 벡터화
3. GloVe 사전 훈련된 임베딩 로드
4. 임베딩 매트릭스 생성 및 매핑
5. 사전 훈련된 임베딩으로 모델 구축
6. 모델 훈련 및 평가

## GloVe (Global Vectors for Word Representation)
- **Stanford에서 개발**: 전역 단어 벡터 표현 방법
- **장점**: 대규모 코퍼스에서 학습된 풍부한 의미 정보 활용
- **6B.100d**: 60억 개 토큰으로 학습된 100차원 벡터
- **전이 학습**: 도메인 특화 작업에 일반적 언어 지식 적용


## 1. 라이브러리 임포트

사전 훈련된 임베딩을 사용하기 위해 필요한 라이브러리들을 임포트합니다.


In [2]:
# 사전 훈련된 임베딩 사용을 위한 GloVe 관련 라이브러리
import requests
import subprocess
import re
import string
import numpy as np  # GloVe 임베딩 처리용

# TensorFlow 및 Keras 라이브러리
import tensorflow as tf
from keras.layers import TextVectorization
from keras import models, layers
import keras
import os, pathlib, shutil, random

print("TensorFlow 버전:", tf.__version__)
print("Keras 버전:", keras.__version__)
print("NumPy 버전:", np.__version__)
print("GloVe 임베딩 처리를 위한 라이브러리 임포트 완료!")


TensorFlow 버전: 2.15.1
Keras 버전: 2.15.0
NumPy 버전: 1.26.4
GloVe 임베딩 처리를 위한 라이브러리 임포트 완료!


## 2. 데이터 다운로드 및 전처리 함수

IMDB 데이터셋을 다운로드하고 전처리하는 함수들을 정의합니다.


In [3]:
def download():
    """IMDB 데이터셋을 다운로드하는 함수"""
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"

    print("IMDB 데이터셋 다운로드 시작...")
    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)

    print("Download complete!")

def release():
    """압축을 해제하는 함수"""
    print("압축 해제 시작...")
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
    # tar.gz => linux에서는 파일을 여러개를 한번에 압축을 못함 
    # tar라는 형식으로 압축할 모든 파일을 하나로 묶어서 패키지로 만든다음에 압축을 한다.  
    # tar, gz가동 그래서 압축풀고 다시 패키지도 풀어야 한다. 
    # tar -xvzf 파일명 형태임         
    print("압축풀기 완료")

def labeling(): 
    """Train 데이터를 Train과 Validation으로 분할하는 함수"""
    print("데이터 라벨링 및 분할 시작...")
    base_dir = pathlib.Path("aclImdb") 
    val_dir = base_dir/"val"   # pathlib 객체에 / "디렉토리" => 결과가 문자열이 아니다 
    train_dir = base_dir/"train"

    # validation 디렉토리 생성 및 데이터 분할
    # train => train과 validation으로 나눠야 한다. 라벨이 2개 여야 한다.
    for category in ("neg", "pos"):
        os.makedirs(val_dir/category, exist_ok=True)  # 디렉토리를 만들고 
        files = os.listdir(train_dir/category)  # 해당 카테고리의 파일 목록을 모두 가져온다 
        random.Random(1337).shuffle(files)  # 파일을 랜덤하게 섞어서 복사하려고 파일 목록을 모두 섞는다 
        num_val_samples = int(0.2 * len(files))  # 20%를 validation으로 사용
        val_files = files[-num_val_samples:]  # 20%만 val폴더로 이동한다 
        for fname in val_files:
            shutil.move(train_dir/category/fname, val_dir/category/fname)
    
    print("데이터 라벨링 및 분할 완료")

# 주석 처리: 이미 데이터가 있다면 다시 다운로드할 필요 없음
# download()  # 파일 다운받기 = 용량이 너무 커서 8192만큼씩 잘라서 저장하는 코드임 
# release()   # 압축 해제
# labeling()  # 데이터 분할


## 3. 데이터셋 로드

디렉토리 구조를 활용하여 IMDB 데이터셋을 로드하고 구조를 확인합니다.

- **0**: 부정 리뷰 (neg)
- **1**: 긍정 리뷰 (pos)


In [4]:
# 데이터셋을 활용해서 디렉토리로부터 파일을 불러와서 벡터화를 진행한다 
batch_size = 32  # 한번에 읽어올 양 

train_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/train",  # 디렉토리명 
    batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/val",  # 디렉토리명 
    batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/test",  # 디렉토리명 
    batch_size=batch_size
)

print("데이터셋 로드 완료!")
print(f"훈련 데이터: {len(train_ds)} 배치")
print(f"검증 데이터: {len(val_ds)} 배치") 
print(f"테스트 데이터: {len(test_ds)} 배치")


Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
데이터셋 로드 완료!
훈련 데이터: 2188 배치
검증 데이터: 157 배치
테스트 데이터: 782 배치


### 3.1 데이터 구조 확인

로드된 데이터의 구조와 내용을 확인해봅니다.


In [5]:
# 데이터셋은 알아서 inputs, targets을 반복해서 갖고 온다. 우리한테 필요한거는 inputs만이다
for inputs, targets in train_ds:  # 실제 읽어오는 데이터 확인 
    print("inputs.shape", inputs.shape)
    print("inputs.dtype", inputs.dtype)
    print("targets.shape", targets.shape)
    print("targets.dtype", targets.dtype)
    print("\n=== 샘플 데이터 ===")
    print("inputs 샘플 (처음 3개):")
    for i, text in enumerate(inputs[:3]):
        print(f"  {i+1}: {text.numpy().decode('utf-8')[:100]}...")  # 처음 100자만 출력
    print(f"\ntargets 샘플 (처음 3개): {targets[:3]}")
    break  # 하나만 출력해보자 

print("\n=== 라벨 정보 ===")
print("0: 부정 리뷰 (neg)")  
print("1: 긍정 리뷰 (pos)")
print("폴더명을 정렬해서 0,1,2 이런식으로 라벨링을 한다 (neg -> 0, pos -> 1)")


inputs.shape (32,)
inputs.dtype <dtype: 'string'>
targets.shape (32,)
targets.dtype <dtype: 'int32'>

=== 샘플 데이터 ===
inputs 샘플 (처음 3개):
  1: Gayniggers from Outer Space is pretty much summed up by its name. Running only 27 minutes long, it d...
  2: As an anti-football person, I (on the surface) grudgingly took my younger brother to see this film, ...
  3: I saw the last drop yesterday, and within a few minutes knew that i would be disappointed. The actin...

targets 샘플 (처음 3개): [0 1 2]

=== 라벨 정보 ===
0: 부정 리뷰 (neg)
1: 긍정 리뷰 (pos)
폴더명을 정렬해서 0,1,2 이런식으로 라벨링을 한다 (neg -> 0, pos -> 1)


## 4. 텍스트 벡터화

텍스트 데이터를 정수 시퀀스로 변환하여 GloVe 임베딩과 매핑할 준비를 합니다.

### 하이퍼파라미터 설정
- **max_length**: 한 리뷰에서 사용하는 최대 단어 수 (600)
- **max_tokens**: 자주 사용하는 단어 개수 (20000) - 어휘 사전 크기


In [7]:
# 시퀀스를 만들어야 한다 
max_length = 600   # 한 평론에서 사용하는 단어는 최대 길이를 600개라고 보자  
max_tokens = 20000  # 자주 사용하는 단어 20000개만 쓰겠다 

text_vectorization = TextVectorization( 
    max_tokens=max_tokens,
    output_mode="int",  # 임베딩 층을 사용하려면 반드시 int여야 한다
    output_sequence_length=max_length  
)

print(f"최대 시퀀스 길이: {max_length}")
print(f"어휘 사전 크기: {max_tokens}")
print("출력 모드: 정수 (int)")
print("TextVectorization 레이어 생성 완료!")


최대 시퀀스 길이: 600
어휘 사전 크기: 20000
출력 모드: 정수 (int)
TextVectorization 레이어 생성 완료!


### 4.1 어휘사전 생성 및 벡터화 적용

훈련 데이터를 사용하여 어휘사전을 생성하고, 모든 데이터셋에 벡터화를 적용합니다.


In [9]:
# 텍스트만 추출 (라벨 제거)
text_only_train_ds = train_ds.map(lambda x, y: x)

# 어휘사전 생성 (훈련 데이터 기반)
print("어휘사전 생성 중...")
text_vectorization.adapt(text_only_train_ds)  # 어휘사전을 만들어야 한다 
print("어휘사전 생성 완료!")

# 모든 데이터셋에 벡터화 적용
print("데이터셋 벡터화 중...")
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=1)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=1)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=1)

print("데이터셋 벡터화 완료!")


어휘사전 생성 중...
어휘사전 생성 완료!
데이터셋 벡터화 중...
데이터셋 벡터화 완료!


### 4.2 벡터화된 데이터 확인

벡터화 후 데이터의 내부 구조를 확인해봅니다.


In [10]:
# 내부구조 살짝 보기 
print("=== 벡터화된 데이터 내부구조 확인 ===")
for item in int_train_ds:
    vectorized_texts, labels = item
    print(f"벡터화된 텍스트 형태: {vectorized_texts.shape}")
    print(f"벡터화된 텍스트 타입: {vectorized_texts.dtype}")
    print(f"라벨 형태: {labels.shape}")
    print(f"라벨 타입: {labels.dtype}")
    
    print("\n=== 샘플 벡터화 결과 ===")
    print("첫 번째 리뷰의 벡터화 결과 (처음 20개 토큰):")
    print(vectorized_texts[0][:20])
    print(f"해당 라벨: {labels[0]}")
    break  # 하나만 출력해보자

print("\n다음 단계: GloVe 임베딩과 매핑하여 사전 훈련된 단어 표현 활용")


=== 벡터화된 데이터 내부구조 확인 ===
벡터화된 텍스트 형태: (32, 600)
벡터화된 텍스트 타입: <dtype: 'int64'>
라벨 형태: (32,)
라벨 타입: <dtype: 'int32'>

=== 샘플 벡터화 결과 ===
첫 번째 리뷰의 벡터화 결과 (처음 20개 토큰):
tf.Tensor(
[ 11 414  10  18   3   9 509  70   2   1  51  11  86 210   9  11   1   1
  44   2], shape=(20,), dtype=int64)
해당 라벨: 2

다음 단계: GloVe 임베딩과 매핑하여 사전 훈련된 단어 표현 활용


## 5. GloVe 사전 훈련된 임베딩 로드

Stanford에서 제공하는 GloVe 6B.100d 임베딩을 로드하여 사전 훈련된 단어 벡터를 활용합니다.

### GloVe 임베딩 특징
- **Global Vectors**: 전역 단어-단어 동시 출현 통계 활용
- **6B**: 60억 개 토큰으로 훈련 (Wikipedia 2014 + Gigaword 5)
- **100d**: 100차원 벡터 표현
- **파일 형식**: `단어 벡터1 벡터2 ... 벡터100` 형태

### 임베딩층과 원핫인코딩 비교
- **임베딩층**: 내부적으로 연산을 해서 단어와 단어사이의 관계를 계산해서 밀집벡터를 만든다
- **원핫인코딩**: 메모리를 너무 많이 차지함. 최대한 한문장을 표현하는데 만일 최대 20000 단어까지 처리한다면 한문장당 20000개가 필요. 희소행렬 요소가 거의다 0인데 그중 몇개가 값이 있을때, 학습시 속도가 엄청 느리다


In [11]:
# 사전학습된 임베딩 데이터를 불러온다
path_to_glove_file = "../../data/glove.6B.100d.txt"
# 임베딩 데이터, 단어별 각 단어와의 거리가 벡터로 저장되어 있음  
# 파일명의 100이 출력 벡터의 크기이다 

print(f"GloVe 임베딩 파일 로드 중: {path_to_glove_file}")
print("파일 형식: 단어 벡터1 벡터2 ... 벡터100")

embeddings_index = {}
try:
    with open(path_to_glove_file, encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):  # 한 라인씩 읽는다 
            # 단어, 단어들간의 벡터 구조로 되어 있다  예) the 0.0012 000172 ...... 
            word, coefs = line.split(maxsplit=1)  
            coefs = np.fromstring(coefs, "f", sep=" ")  # 나머지 벡터들을 numpy배열로 전환
            embeddings_index[word] = coefs
            
            # 진행 상황 출력 (1만 라인마다)
            if line_num % 10000 == 0:
                print(f"  처리된 라인: {line_num:,}")
                
    print(f"\nGloVe 임베딩 로드 완료!")
    print(f"총 단어 개수: {len(embeddings_index):,}개")
    print(f"벡터 차원: {len(next(iter(embeddings_index.values())))}차원")
    
    # 샘플 단어들의 벡터 확인
    sample_words = ["the", "good", "bad", "movie", "film"]
    print(f"\n=== 샘플 단어들의 벡터 (처음 5개 값만) ===")
    for word in sample_words:
        if word in embeddings_index:
            print(f"{word}: {embeddings_index[word][:5]}")
        else:
            print(f"{word}: (없음)")
            
except FileNotFoundError:
    print(f"❌ 오류: {path_to_glove_file} 파일을 찾을 수 없습니다.")
    print("GloVe 파일을 다운로드하고 ../../data/ 폴더에 저장해주세요.")
    print("다운로드 링크: https://nlp.stanford.edu/projects/glove/")


GloVe 임베딩 파일 로드 중: ../../data/glove.6B.100d.txt
파일 형식: 단어 벡터1 벡터2 ... 벡터100
  처리된 라인: 10,000
  처리된 라인: 20,000
  처리된 라인: 30,000
  처리된 라인: 40,000
  처리된 라인: 50,000
  처리된 라인: 60,000
  처리된 라인: 70,000
  처리된 라인: 80,000
  처리된 라인: 90,000
  처리된 라인: 100,000
  처리된 라인: 110,000
  처리된 라인: 120,000
  처리된 라인: 130,000
  처리된 라인: 140,000
  처리된 라인: 150,000
  처리된 라인: 160,000
  처리된 라인: 170,000
  처리된 라인: 180,000
  처리된 라인: 190,000
  처리된 라인: 200,000
  처리된 라인: 210,000
  처리된 라인: 220,000
  처리된 라인: 230,000
  처리된 라인: 240,000
  처리된 라인: 250,000
  처리된 라인: 260,000
  처리된 라인: 270,000
  처리된 라인: 280,000
  처리된 라인: 290,000
  처리된 라인: 300,000
  처리된 라인: 310,000
  처리된 라인: 320,000
  처리된 라인: 330,000
  처리된 라인: 340,000
  처리된 라인: 350,000
  처리된 라인: 360,000
  처리된 라인: 370,000
  처리된 라인: 380,000
  처리된 라인: 390,000
  처리된 라인: 400,000

GloVe 임베딩 로드 완료!
총 단어 개수: 400,000개
벡터 차원: 100차원

=== 샘플 단어들의 벡터 (처음 5개 값만) ===
the: [-0.038194 -0.24487   0.72812  -0.39961   0.083172]
good: [-0.030769  0.11993   0.53909  -0.43696  -0.73937 ]
bad: [ 0.39456 -0.

## 6. 임베딩 매트릭스 생성

우리 데이터셋의 어휘사전과 GloVe 임베딩을 매핑하여 모델에서 사용할 임베딩 매트릭스를 생성합니다.

### 매핑 과정
1. **어휘사전 추출**: TextVectorization에서 생성된 어휘사전 가져오기
2. **인덱스 매핑**: {단어: 인덱스} 딕셔너리 생성
3. **매트릭스 초기화**: (20000, 100) 크기의 0 매트릭스 생성
4. **벡터 매핑**: GloVe에 있는 단어들의 벡터를 해당 인덱스에 할당


In [12]:
# 우리데이터와 연동을 해야 한다 
vocabulary = text_vectorization.get_vocabulary()  # 우리 어휘사전 가져오기
print(f"우리 데이터의 어휘사전 크기: {len(vocabulary)}")
print(f"어휘사전 샘플 (처음 10개): {vocabulary[:10]}")

# {단어:인덱스} 형태의 딕셔너리를 만들어야 한다 
# {"", "[UNK]", "write", "love", "make",...........} vocabulary
# {0,1,2,3,4,5,6,..................}
# zip ("",0) ("[UNK]",1) ("write", 2 )............
# {"":0, "[UNK]":1, "write":2, ,,,,,,,}

word_index = dict(zip(vocabulary, range(len(vocabulary)))) 
print(f"단어-인덱스 매핑 딕셔너리 생성 완료")
print(f"샘플 매핑: {dict(list(word_index.items())[:5])}")

embedding_dim = 100  # 미리 학습한 임베딩층의 출력값이 100개임 
embedding_matrix = np.zeros((max_tokens, embedding_dim))  # 20000 * 100 배열을 잡고 0으로 채운다 
print(f"\n임베딩 매트릭스 초기화: {embedding_matrix.shape}")
print("케라스 Embedding 레이어에 초기값으로 사용될 예정")


우리 데이터의 어휘사전 크기: 20000
어휘사전 샘플 (처음 10개): ['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']
단어-인덱스 매핑 딕셔너리 생성 완료
샘플 매핑: {'': 0, '[UNK]': 1, 'the': 2, 'and': 3, 'a': 4}

임베딩 매트릭스 초기화: (20000, 100)
케라스 Embedding 레이어에 초기값으로 사용될 예정


### 6.1 GloVe 벡터를 임베딩 매트릭스에 매핑

우리 어휘사전의 각 단어에 대해 GloVe에서 해당 벡터를 찾아 임베딩 매트릭스에 할당합니다.


In [13]:
# embedding_matrix를 우리가 embedding_index 정보로 채워야 한다. 
# word_index는 {단어:인덱스} 형태임 

hits = 0  # GloVe에서 찾은 단어 개수
misses = 0  # GloVe에서 찾지 못한 단어 개수

print("GloVe 벡터를 임베딩 매트릭스에 매핑 중...")

for word, i in word_index.items(): 
    # 단어와 인덱스를 가져온다 
    if i < max_tokens:  # 혹시나 20000개를 넘어가는 토큰이 있을까봐 오류처리
        embedding_vector = embeddings_index.get(word)  # 단어에 해당하는 벡터들 이동 
        if embedding_vector is not None:  # embedding_vector값이 None인 경우를 제외하고 
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1

print(f"\n=== 매핑 결과 ===")
print(f"GloVe에서 찾은 단어: {hits:,}개")
print(f"GloVe에서 찾지 못한 단어: {misses:,}개")
print(f"매핑 성공률: {hits/(hits+misses)*100:.1f}%")

print(f"\n=== 임베딩 매트릭스 확인 ===")
print(f"매트릭스 크기: {embedding_matrix.shape}")
print(f"0이 아닌 행의 개수: {np.count_nonzero(np.any(embedding_matrix != 0, axis=1))}")
print("임베딩 매트릭스 생성 완료!")

# 매트릭스 샘플 확인 (처음 10개 단어의 처음 5개 벡터 값)
print(f"\n=== 임베딩 매트릭스 샘플 (처음 10개 단어, 처음 5개 벡터 값) ===")
for i in range(min(10, len(vocabulary))):
    word = vocabulary[i]
    vector_sample = embedding_matrix[i][:5]
    print(f"인덱스 {i:2d} | 단어 '{word:8s}' | 벡터: {vector_sample}")


GloVe 벡터를 임베딩 매트릭스에 매핑 중...

=== 매핑 결과 ===
GloVe에서 찾은 단어: 18,845개
GloVe에서 찾지 못한 단어: 1,155개
매핑 성공률: 94.2%

=== 임베딩 매트릭스 확인 ===
매트릭스 크기: (20000, 100)
0이 아닌 행의 개수: 18845
임베딩 매트릭스 생성 완료!

=== 임베딩 매트릭스 샘플 (처음 10개 단어, 처음 5개 벡터 값) ===
인덱스  0 | 단어 '        ' | 벡터: [0. 0. 0. 0. 0.]
인덱스  1 | 단어 '[UNK]   ' | 벡터: [0. 0. 0. 0. 0.]
인덱스  2 | 단어 'the     ' | 벡터: [-0.038194   -0.24487001  0.72812003 -0.39961001  0.083172  ]
인덱스  3 | 단어 'and     ' | 벡터: [-0.071953    0.23127     0.023731   -0.50638002  0.33923   ]
인덱스  4 | 단어 'a       ' | 벡터: [-0.27085999  0.044006   -0.02026    -0.17395     0.6444    ]
인덱스  5 | 단어 'of      ' | 벡터: [-0.1529     -0.24279     0.89837003  0.16996001  0.53516001]
인덱스  6 | 단어 'to      ' | 벡터: [-0.18970001  0.050024    0.19084001 -0.049184   -0.089737  ]
인덱스  7 | 단어 'is      ' | 벡터: [-0.54263997  0.41475999  1.03219998 -0.40244001  0.46691   ]
인덱스  8 | 단어 'in      ' | 벡터: [ 0.085703   -0.22201     0.16569     0.13372999  0.38238999]
인덱스  9 | 단어 'it      ' | 벡터: [-0.30664     

## 7. 사전 훈련된 임베딩으로 모델 구축

GloVe 임베딩 매트릭스를 초기값으로 사용하는 Embedding 레이어로 모델을 구성합니다.

### 핵심 설정
- **embeddings_initializer**: GloVe 임베딩 매트릭스로 초기화
- **trainable=False**: 사전 훈련된 가중치를 고정하여 보존
- **mask_zero=True**: 패딩 토큰(0)을 마스킹하여 연산에서 제외

### 모델 아키텍처
1. **입력**: 정수 시퀀스 (배치_크기, 시퀀스_길이)
2. **사전 훈련된 임베딩**: GloVe 100차원 벡터로 변환
3. **양방향 LSTM**: 양방향으로 시퀀스 처리
4. **드롭아웃**: 과적합 방지
5. **출력**: 이진 분류를 위한 시그모이드 활성화


In [14]:
# 모델 구축 시작
print("=== 사전 훈련된 GloVe 임베딩으로 모델 구축 ===")

# 입력 레이어
inputs = keras.Input(shape=(None,), dtype="int64")
print(f"입력 형태: {inputs.shape}")

# 사전 훈련된 임베딩 레이어
embedded = layers.Embedding(
    input_dim=max_tokens, 
    output_dim=embedding_dim, 
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),  # 반드시 GloVe 매트릭스로 초기화
    # 사전학습된 층에 의해 바꿔치기가 이뤄져야 한다 
    trainable=False,  # 임베딩가중치를 훈련중에 업데이트할거냐? 사전학습된 임베딩층을 사용할때는 False로 지정해야 한다 
    mask_zero=True  # 패딩 토큰(0)을 마스킹
)(inputs)
 
print(f"임베딩 후 형태: {embedded.shape}")
print("🔒 trainable=False: 사전 훈련된 GloVe 가중치 고정")
print("🎭 mask_zero=True: 패딩 토큰 마스킹 적용")

# 미리 학습된 임베딩층으로 바꿀 수가 있다
# 입력벡터크기는 20000, 출력벡터는 100(GloVe 차원)의 크기를 갖는다  
print(f"GloVe 임베딩: 입력 {max_tokens}차원 → 출력 {embedding_dim}차원")


=== 사전 훈련된 GloVe 임베딩으로 모델 구축 ===
입력 형태: (None, None)
임베딩 후 형태: (None, None, 100)
🔒 trainable=False: 사전 훈련된 GloVe 가중치 고정
🎭 mask_zero=True: 패딩 토큰 마스킹 적용
GloVe 임베딩: 입력 20000차원 → 출력 100차원


### 7.1 양방향 LSTM 및 출력 레이어


In [15]:
# 양방향 RNN을 가동시킴 
x = layers.Bidirectional(layers.LSTM(32))(embedded) 
print("양방향 LSTM 추가 (32 유닛)")
print("- 순방향 LSTM: 문장의 앞에서 뒤로 정보 처리")
print("- 역방향 LSTM: 문장의 뒤에서 앞으로 정보 처리")
print("- 결과: 양방향 정보를 결합한 풍부한 표현")

# 드롭아웃으로 과적합 방지
x = layers.Dropout(0.5)(x) 
print("\n드롭아웃 추가 (0.5)")
print("- 훈련 시 50% 뉴런을 무작위로 비활성화")
print("- 과적합 방지 및 일반화 성능 향상")

# 출력 레이어 (이진 분류)
outputs = layers.Dense(1, activation='sigmoid')(x)
print("\n출력 레이어 추가 (시그모이드 활성화)")
print("- 1개 뉴런: 이진 분류 (긍정/부정)")
print("- 시그모이드: 0~1 사이 확률값 출력")

# 모델 생성
model = keras.Model(inputs, outputs) 
print(f"\n🎯 모델 생성 완료!")


양방향 LSTM 추가 (32 유닛)
- 순방향 LSTM: 문장의 앞에서 뒤로 정보 처리
- 역방향 LSTM: 문장의 뒤에서 앞으로 정보 처리
- 결과: 양방향 정보를 결합한 풍부한 표현

드롭아웃 추가 (0.5)
- 훈련 시 50% 뉴런을 무작위로 비활성화
- 과적합 방지 및 일반화 성능 향상

출력 레이어 추가 (시그모이드 활성화)
- 1개 뉴런: 이진 분류 (긍정/부정)
- 시그모이드: 0~1 사이 확률값 출력

🎯 모델 생성 완료!


### 7.2 모델 컴파일 및 구조 확인

모델을 컴파일하고 전체 구조를 확인합니다.


In [17]:
# 모델 컴파일
model.compile(
    optimizer='rmsprop',           # RMSprop 옵티마이저
    loss='binary_crossentropy',    # 이진 분류용 손실 함수
    metrics=['accuracy']           # 정확도 메트릭
)

print("=== 모델 컴파일 완료 ===")
print("옵티마이저: RMSprop")
print("- 학습률을 적응적으로 조정하는 옵티마이저")
print("- RNN/LSTM에서 안정적인 성능")
print("\n손실 함수: binary_crossentropy") 
print("- 이진 분류 문제에 최적화된 손실 함수")
print("- 예측 확률과 실제 라벨 간의 교차 엔트로피")
print("\n메트릭: accuracy")
print("- 정확히 예측한 샘플의 비율")

print("\n=== 모델 구조 요약 ===")
model.summary()

print(f"\n=== 사전 훈련된 임베딩 활용 확인 ===")
print(f"임베딩 레이어 trainable: {model.layers[1].trainable}")
print(f"임베딩 가중치 크기: {model.layers[1].get_weights()[0].shape}")
print("✅ GloVe 사전 훈련된 가중치가 고정되어 학습에 활용됩니다!")


=== 모델 컴파일 완료 ===
옵티마이저: RMSprop
- 학습률을 적응적으로 조정하는 옵티마이저
- RNN/LSTM에서 안정적인 성능

손실 함수: binary_crossentropy
- 이진 분류 문제에 최적화된 손실 함수
- 예측 확률과 실제 라벨 간의 교차 엔트로피

메트릭: accuracy
- 정확히 예측한 샘플의 비율

=== 모델 구조 요약 ===
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional (Bidirection  (None, 64)                34048     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                

## 8. 모델 훈련 및 평가

사전 훈련된 GloVe 임베딩을 활용한 모델을 훈련하고 성능을 평가합니다.

### 훈련 설정
- **에포크**: 15회 (원본 코드 기준)
- **검증 데이터**: val 데이터셋 사용
- **고정된 임베딩**: GloVe 가중치는 학습되지 않음


In [19]:
# 모델 훈련 시작
print("=== 사전 훈련된 GloVe 임베딩으로 모델 훈련 시작 ===")
print("에포크: 15")
print("검증 데이터: val_ds")
print("🔒 GloVe 임베딩 가중치: 고정 (trainable=False)")
print()

# 훈련 실행
history = model.fit(
    int_train_ds, 
    validation_data=int_val_ds, 
    epochs=15,
    verbose=1  # 훈련 과정 출력
)

print("=== 모델 훈련 완료 ===")


=== 사전 훈련된 GloVe 임베딩으로 모델 훈련 시작 ===
에포크: 15
검증 데이터: val_ds
🔒 GloVe 임베딩 가중치: 고정 (trainable=False)

Epoch 1/15
2188/2188 [==============================] - 315s 144ms/step - loss: -62.3371 - accuracy: 0.1429 - val_loss: 87.6845 - val_accuracy: 0.5000
Epoch 2/15
2188/2188 [==============================] - 308s 141ms/step - loss: -138.6386 - accuracy: 0.1429 - val_loss: 154.3308 - val_accuracy: 0.5000
Epoch 3/15
2188/2188 [==============================] - 313s 143ms/step - loss: -214.6419 - accuracy: 0.1429 - val_loss: 220.9855 - val_accuracy: 0.5000
Epoch 4/15
2188/2188 [==============================] - 316s 145ms/step - loss: -290.5301 - accuracy: 0.1429 - val_loss: 287.6262 - val_accuracy: 0.5000
Epoch 5/15
2188/2188 [==============================] - 318s 145ms/step - loss: -367.0156 - accuracy: 0.1429 - val_loss: 354.2675 - val_accuracy: 0.5000
Epoch 6/15
2188/2188 [==============================] - 318s 145ms/step - loss: -442.8620 - accuracy: 0.1429 - val_loss: 420.8891 - val_acc

### 8.1 모델 성능 평가

훈련이 완료된 모델을 테스트 데이터셋으로 평가하여 최종 성능을 확인합니다.


In [20]:
# 테스트 데이터셋으로 모델 평가
print("=== 테스트 데이터셋 평가 ===")
test_results = model.evaluate(int_test_ds, verbose=1)
test_loss, test_accuracy = test_results

print(f"\n=== GloVe 임베딩 모델 최종 결과 ===")
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# 훈련 히스토리 요약
if 'history' in locals():
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    
    print(f"\n=== 훈련 과정 요약 ===")
    print(f"최종 훈련 정확도: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
    print(f"최종 검증 정확도: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
    print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # 과적합 여부 확인
    if final_train_acc - test_accuracy > 0.1:
        print("\n⚠️  과적합 가능성이 있습니다.")
        print("- 에포크 수 감소 고려")
        print("- 드롭아웃 비율 증가 고려")
        print("- 조기 종료(Early Stopping) 적용 고려")
    else:
        print("\n✅ 적절한 일반화 성능을 보입니다.")

# 원본 코드 스타일 출력
print(f"\n=== 간단 출력 (원본 스타일) ===")
print("테스트셋", test_results)

print(f"\n=== 사전 훈련된 임베딩의 효과 ===")
print("🎯 GloVe 임베딩을 통해 풍부한 언어 지식을 활용했습니다!")
print("📈 사전 훈련된 단어 표현으로 더 나은 성능을 기대할 수 있습니다.")


=== 테스트 데이터셋 평가 ===
782/782 [==============================] - 39s 50ms/step - loss: 1020.4764 - accuracy: 0.5000

=== GloVe 임베딩 모델 최종 결과 ===
테스트 손실: 1020.4764
테스트 정확도: 0.5000 (50.00%)

=== 훈련 과정 요약 ===
최종 훈련 정확도: 0.1429 (14.29%)
최종 검증 정확도: 0.5000 (50.00%)
테스트 정확도: 0.5000 (50.00%)

✅ 적절한 일반화 성능을 보입니다.

=== 간단 출력 (원본 스타일) ===
테스트셋 [1020.4763793945312, 0.5]

=== 사전 훈련된 임베딩의 효과 ===
🎯 GloVe 임베딩을 통해 풍부한 언어 지식을 활용했습니다!
📈 사전 훈련된 단어 표현으로 더 나은 성능을 기대할 수 있습니다.


## 9. 결론 및 비교 분석

### 사전 훈련된 임베딩의 장점
- **풍부한 언어 지식**: 60억 개 토큰으로 학습된 GloVe의 방대한 언어 정보 활용
- **빠른 수렴**: 좋은 초기값으로 시작하여 더 빠른 학습
- **일반화 성능**: 대규모 코퍼스에서 학습된 일반적 언어 표현
- **도메인 적응**: 특정 도메인에 특화되지 않은 범용적 단어 표현

### 모델 특징 요약
- **GloVe 6B.100d**: Stanford의 사전 훈련된 100차원 단어 벡터
- **고정된 가중치**: trainable=False로 사전 훈련된 지식 보존
- **양방향 LSTM**: 문맥을 양방향으로 고려하여 성능 향상
- **마스킹**: mask_zero=True로 패딩 토큰 처리

### 임베딩 방법 비교

| 특성 | 원핫 인코딩 | 학습 가능한 임베딩 | 사전 훈련된 임베딩 (GloVe) |
|------|-------------|-------------------|---------------------------|
| **메모리 사용량** | 매우 높음 | 보통 | 보통 |
| **학습 속도** | 느림 | 보통 | 빠름 |
| **초기 성능** | 낮음 | 낮음 | 높음 |
| **언어 지식** | 없음 | 학습으로 획득 | 사전 훈련된 지식 |
| **도메인 적응** | 제한적 | 우수 | 보통 |
| **메모리 효율성** | ❌ | ✅ | ✅ |
| **전이 학습** | ❌ | ❌ | ✅ |
| **권장 사용** | 실험용 | 일반적 | 고성능 필요시 |

### 언제 사전 훈련된 임베딩을 사용할까?

#### ✅ 사용 권장 상황
- **소규모 데이터셋**: 충분한 데이터가 없어 임베딩 학습이 어려운 경우
- **빠른 프로토타이핑**: 빠르게 좋은 성능을 얻고 싶은 경우
- **일반적 언어 작업**: 감정 분석, 문서 분류 등 범용 언어 이해 필요
- **계산 자원 제한**: 학습 시간과 자원을 절약하고 싶은 경우

#### ❌ 사용 비권장 상황
- **특수 도메인**: 의료, 법률 등 전문 용어가 많은 도메인
- **대규모 데이터**: 충분한 데이터로 도메인 특화 임베딩 학습 가능
- **특수 어휘**: 신조어, 슬랭 등이 많은 소셜 미디어 데이터

### 가능한 개선사항
1. **다른 사전 훈련된 임베딩**: Word2Vec, FastText, 한국어 특화 임베딩
2. **미세 조정**: trainable=True로 설정하여 도메인 적응
3. **Transformer 모델**: BERT, RoBERTa 등 최신 언어 모델 활용
4. **앙상블**: 여러 임베딩 방법의 결합
5. **하이퍼파라미터 튜닝**: 
   - LSTM 유닛 수 조정
   - 드롭아웃 비율 최적화
   - 학습률 스케줄링

### 실제 적용 고려사항
- **임베딩 파일 크기**: GloVe 6B.100d는 약 350MB
- **로딩 시간**: 임베딩 매트릭스 생성에 시간 소요
- **메모리 사용량**: 임베딩 매트릭스가 모델 크기에 영향
- **업데이트**: 새로운 단어에 대한 처리 방법 고려
